# Instagram Influencers Analysis
**Author:** [Your Name]  
**Date:** December 2024

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
import warnings
import os

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

os.makedirs('visualizations', exist_ok=True)
os.makedirs('data', exist_ok=True)

print("Libraries loaded!")

## 1. Load Data

In [ ]:
df = pd.read_csv('top_insta_influencers_data.csv')
print(f"Loaded {len(df)} records")
df.head(10)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

## 2. Data Cleaning

In [ ]:
# convert columns like '475.8m' to numbers
cols_to_fix = ['posts', 'followers', 'avg_likes', '60_day_eng_rate', 'new_post_avg_like', 'total_likes']

for col in cols_to_fix:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('b', 'e9', regex=False)
    df[col] = df[col].str.replace('m', 'e6', regex=False)
    df[col] = df[col].str.replace('k', 'e3', regex=False)
    df[col] = df[col].str.replace('%', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['country'] = df['country'].fillna('Unknown')
df['60_day_eng_rate'] = df['60_day_eng_rate'].fillna(df['60_day_eng_rate'].median())

print("Done!")
print(f"Cristiano followers: {df.loc[0, 'followers']:,.0f}")

In [ ]:
df.to_csv('data/cleaned_data.csv', index=False)
print("Saved cleaned data")

In [ ]:
df.describe()

## 3. Basic Stats

In [ ]:
print(f"Total influencers: {len(df)}")
print(f"Countries: {df['country'].nunique()}")
print(f"Avg followers: {df['followers'].mean():,.0f}")
print(f"Avg engagement: {df['60_day_eng_rate'].mean():.2f}%")
print(f"Influence score range: {df['influence_score'].min()} - {df['influence_score'].max()}")

## 4. Visualizations

In [ ]:
# Top 15 by followers
fig, ax = plt.subplots(figsize=(12, 8))
top15 = df.nlargest(15, 'followers')
colors = plt.cm.Blues(np.linspace(0.4, 0.9, 15))[::-1]
bars = ax.barh(top15['channel_info'], top15['followers']/1e6, color=colors)
ax.set_xlabel('Followers (Millions)', fontsize=12)
ax.set_title('Top 15 Instagram Influencers by Followers', fontsize=14, fontweight='bold')
ax.invert_yaxis()
for i, (bar, val) in enumerate(zip(bars, top15['followers']/1e6)):
    ax.text(val + 5, bar.get_y() + bar.get_height()/2, f'{val:.1f}M', va='center', fontsize=9)
plt.tight_layout()
plt.savefig('visualizations/01_top15_followers.png', dpi=150)
plt.show()

In [ ]:
# Country distribution
fig, ax = plt.subplots(figsize=(12, 6))
country_counts = df['country'].value_counts().head(12)
colors = plt.cm.Spectral(np.linspace(0, 1, 12))
bars = ax.bar(country_counts.index, country_counts.values, color=colors, edgecolor='black', linewidth=0.5)
ax.set_xlabel('Country', fontsize=12)
ax.set_ylabel('Number of Influencers', fontsize=12)
ax.set_title('Influencers by Country', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, int(bar.get_height()), ha='center', fontsize=9)
plt.tight_layout()
plt.savefig('visualizations/02_country_distribution.png', dpi=150)
plt.show()

In [ ]:
# Followers vs Engagement
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(df['followers']/1e6, df['60_day_eng_rate'], 
                     c=df['influence_score'], cmap='viridis', 
                     s=80, alpha=0.6, edgecolors='white', linewidth=0.5)
ax.set_xlabel('Followers (Millions)', fontsize=12)
ax.set_ylabel('Engagement Rate (%)', fontsize=12)
ax.set_title('Followers vs Engagement Rate', fontsize=14, fontweight='bold')
cbar = plt.colorbar(scatter)
cbar.set_label('Influence Score')
plt.tight_layout()
plt.savefig('visualizations/03_followers_vs_engagement.png', dpi=150)
plt.show()

In [ ]:
# Influence score distribution
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(df['influence_score'], bins=25, color='steelblue', edgecolor='black', alpha=0.8)
ax.axvline(df['influence_score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["influence_score"].mean():.1f}')
ax.axvline(df['influence_score'].median(), color='orange', linestyle='--', linewidth=2, label=f'Median: {df["influence_score"].median():.1f}')
ax.set_xlabel('Influence Score', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Distribution of Influence Scores', fontsize=14, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.savefig('visualizations/04_influence_score_dist.png', dpi=150)
plt.show()

In [ ]:
# Correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))
numeric_cols = ['influence_score', 'posts', 'followers', 'avg_likes', '60_day_eng_rate', 'new_post_avg_like', 'total_likes']
corr = df[numeric_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', center=0, square=True, linewidths=0.5, ax=ax)
ax.set_title('Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('visualizations/05_correlation_matrix.png', dpi=150)
plt.show()

In [ ]:
# Top 10 by engagement
fig, ax = plt.subplots(figsize=(10, 6))
top_eng = df.nlargest(10, '60_day_eng_rate')
colors = plt.cm.Greens(np.linspace(0.4, 0.9, 10))[::-1]
bars = ax.barh(top_eng['channel_info'], top_eng['60_day_eng_rate'], color=colors)
ax.set_xlabel('Engagement Rate (%)', fontsize=12)
ax.set_title('Top 10 by Engagement Rate', fontsize=14, fontweight='bold')
ax.invert_yaxis()
for bar, val in zip(bars, top_eng['60_day_eng_rate']):
    ax.text(val + 0.3, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center', fontsize=9)
plt.tight_layout()
plt.savefig('visualizations/06_top_engagement.png', dpi=150)
plt.show()

In [ ]:
# Engagement by follower tier
df['follower_tier'] = pd.cut(df['followers'], 
                              bins=[0, 50e6, 100e6, 200e6, 500e6],
                              labels=['<50M', '50-100M', '100-200M', '200M+'])

fig, ax = plt.subplots(figsize=(8, 6))
tier_eng = df.groupby('follower_tier')['60_day_eng_rate'].mean()
colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c']
bars = ax.bar(tier_eng.index.astype(str), tier_eng.values, color=colors, edgecolor='black')
ax.set_xlabel('Follower Tier', fontsize=12)
ax.set_ylabel('Avg Engagement Rate (%)', fontsize=12)
ax.set_title('Engagement Rate by Follower Tier', fontsize=14, fontweight='bold')
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, f'{bar.get_height():.2f}%', ha='center', fontsize=10)
plt.tight_layout()
plt.savefig('visualizations/07_engagement_by_tier.png', dpi=150)
plt.show()

In [ ]:
# Posts vs Total Likes
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(df['posts'], df['total_likes']/1e9, alpha=0.6, s=60, c='coral', edgecolors='white')
ax.set_xlabel('Number of Posts', fontsize=12)
ax.set_ylabel('Total Likes (Billions)', fontsize=12)
ax.set_title('Posts vs Total Likes', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('visualizations/08_posts_vs_likes.png', dpi=150)
plt.show()

## 5. Feature Engineering

In [ ]:
df['likes_per_follower'] = df['total_likes'] / df['followers']
df['likes_per_post'] = df['total_likes'] / (df['posts'] + 1)
df['avg_likes_pct'] = (df['avg_likes'] / df['followers']) * 100
df['log_followers'] = np.log10(df['followers'])

df['eng_category'] = pd.cut(df['60_day_eng_rate'], bins=[0, 1, 3, 100], labels=['Low', 'Medium', 'High'])

le = LabelEncoder()
df['country_code'] = le.fit_transform(df['country'])

print("Features created:")
print("- likes_per_follower")
print("- likes_per_post")
print("- avg_likes_pct")
print("- log_followers")
print("- eng_category")
print("- country_code")

In [ ]:
df.to_csv('data/featured_data.csv', index=False)
print("Saved featured data")

In [ ]:
df['eng_category'].value_counts()

## 6. Machine Learning - Regression

In [ ]:
features = ['followers', 'avg_likes', '60_day_eng_rate', 'new_post_avg_like', 'likes_per_follower', 'country_code']
X = df[features].fillna(0)
y = df['influence_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train_sc, y_train)
y_pred_lr = lr.predict(X_test_sc)
r2_lr = r2_score(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

print(f"Linear Regression -> R2: {r2_lr:.4f}, RMSE: {rmse_lr:.2f}")

In [ ]:
# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
rf.fit(X_train_sc, y_train)
y_pred_rf = rf.predict(X_test_sc)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))

print(f"Random Forest -> R2: {r2_rf:.4f}, RMSE: {rmse_rf:.2f}")

In [ ]:
# Plot results
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(y_test, y_pred_lr, alpha=0.6, c='steelblue', edgecolors='white')
axes[0].plot([20, 95], [20, 95], 'r--', linewidth=2)
axes[0].set_xlabel('Actual')
axes[0].set_ylabel('Predicted')
axes[0].set_title(f'Linear Regression (R² = {r2_lr:.3f})')

axes[1].scatter(y_test, y_pred_rf, alpha=0.6, c='seagreen', edgecolors='white')
axes[1].plot([20, 95], [20, 95], 'r--', linewidth=2)
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
axes[1].set_title(f'Random Forest (R² = {r2_rf:.3f})')

plt.suptitle('Regression: Actual vs Predicted Influence Score', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('visualizations/09_regression_results.png', dpi=150)
plt.show()

In [ ]:
# Feature importance
fig, ax = plt.subplots(figsize=(8, 5))
importance = pd.Series(rf.feature_importances_, index=features).sort_values()
importance.plot(kind='barh', color='teal', ax=ax)
ax.set_xlabel('Importance')
ax.set_title('Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('visualizations/10_feature_importance.png', dpi=150)
plt.show()

## 7. Machine Learning - Classification

In [ ]:
df_class = df.dropna(subset=['eng_category'])
X_c = df_class[['followers', 'influence_score', 'country_code', 'log_followers', 'avg_likes_pct']].fillna(0)
y_c = df_class['eng_category']

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c, test_size=0.2, random_state=42, stratify=y_c)

scaler_c = StandardScaler()
X_train_c_sc = scaler_c.fit_transform(X_train_c)
X_test_c_sc = scaler_c.transform(X_test_c)

print(f"Train: {len(X_train_c)}, Test: {len(X_test_c)}")

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42)
clf.fit(X_train_c_sc, y_train_c)
y_pred_c = clf.predict(X_test_c_sc)

acc = accuracy_score(y_test_c, y_pred_c)
print(f"Accuracy: {acc*100:.1f}%")

In [ ]:
print(classification_report(y_test_c, y_pred_c))

In [ ]:
# Confusion matrix
fig, ax = plt.subplots(figsize=(7, 6))
cm = confusion_matrix(y_test_c, y_pred_c, labels=['Low', 'Medium', 'High'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Low', 'Medium', 'High'], 
            yticklabels=['Low', 'Medium', 'High'], ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title(f'Confusion Matrix (Accuracy: {acc*100:.1f}%)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('visualizations/11_confusion_matrix.png', dpi=150)
plt.show()

## 8. Dashboard

In [ ]:
fig = plt.figure(figsize=(20, 16))
fig.suptitle('Instagram Influencers Analysis Dashboard', fontsize=24, fontweight='bold', y=0.98)

# 1. Top 10 followers
ax1 = fig.add_subplot(3, 3, 1)
top10 = df.nlargest(10, 'followers')
ax1.barh(top10['channel_info'], top10['followers']/1e6, color='#3498db')
ax1.set_xlabel('Followers (M)')
ax1.set_title('Top 10 by Followers', fontweight='bold')
ax1.invert_yaxis()

# 2. Country pie
ax2 = fig.add_subplot(3, 3, 2)
top_countries = df['country'].value_counts().head(8)
ax2.pie(top_countries, labels=top_countries.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('Top Countries', fontweight='bold')

# 3. Engagement by tier
ax3 = fig.add_subplot(3, 3, 3)
tier_data = df.groupby('follower_tier')['60_day_eng_rate'].mean()
ax3.bar(tier_data.index.astype(str), tier_data.values, color=['#27ae60', '#2980b9', '#8e44ad', '#c0392b'])
ax3.set_xlabel('Tier')
ax3.set_ylabel('Avg Eng Rate %')
ax3.set_title('Engagement by Follower Tier', fontweight='bold')

# 4. Scatter
ax4 = fig.add_subplot(3, 3, 4)
ax4.scatter(df['followers']/1e6, df['60_day_eng_rate'], c=df['influence_score'], cmap='viridis', alpha=0.6, s=40)
ax4.set_xlabel('Followers (M)')
ax4.set_ylabel('Engagement %')
ax4.set_title('Followers vs Engagement', fontweight='bold')

# 5. Influence hist
ax5 = fig.add_subplot(3, 3, 5)
ax5.hist(df['influence_score'], bins=20, color='#9b59b6', edgecolor='black', alpha=0.8)
ax5.axvline(df['influence_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["influence_score"].mean():.1f}')
ax5.set_xlabel('Score')
ax5.set_title('Influence Score Distribution', fontweight='bold')
ax5.legend(fontsize=8)

# 6. Correlation
ax6 = fig.add_subplot(3, 3, 6)
small_corr = df[['influence_score', 'followers', 'avg_likes', '60_day_eng_rate']].corr()
sns.heatmap(small_corr, annot=True, fmt='.2f', cmap='coolwarm', ax=ax6, cbar=False)
ax6.set_title('Key Correlations', fontweight='bold')

# 7. Top engagement
ax7 = fig.add_subplot(3, 3, 7)
top_eng = df.nlargest(8, '60_day_eng_rate')
ax7.barh(top_eng['channel_info'], top_eng['60_day_eng_rate'], color='#16a085')
ax7.set_xlabel('Engagement Rate %')
ax7.set_title('Top 8 by Engagement', fontweight='bold')
ax7.invert_yaxis()

# 8. Regression
ax8 = fig.add_subplot(3, 3, 8)
ax8.scatter(y_test, y_pred_rf, alpha=0.6, c='#e74c3c', edgecolors='white')
ax8.plot([20, 95], [20, 95], 'k--', linewidth=1.5)
ax8.set_xlabel('Actual')
ax8.set_ylabel('Predicted')
ax8.set_title(f'ML Regression (R²={r2_rf:.3f})', fontweight='bold')

# 9. Classification
ax9 = fig.add_subplot(3, 3, 9)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['Low', 'Med', 'High'], 
            yticklabels=['Low', 'Med', 'High'], ax=ax9, cbar=False)
ax9.set_xlabel('Predicted')
ax9.set_ylabel('Actual')
ax9.set_title(f'Classification ({acc*100:.0f}% acc)', fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('visualizations/12_dashboard.png', dpi=200)
plt.show()

## 9. Key Findings

In [ ]:
high_f = df[df['followers'] > 100e6]['60_day_eng_rate'].mean()
low_f = df[df['followers'] < 50e6]['60_day_eng_rate'].mean()

print("="*50)
print("KEY FINDINGS")
print("="*50)
print(f"""
1. ENGAGEMENT PARADOX
   - High followers (>100M): {high_f:.2f}% avg engagement
   - Lower followers (<50M): {low_f:.2f}% avg engagement
   - Smaller accounts have ~{low_f/high_f:.1f}x higher engagement!

2. TOP PERFORMERS
   - Most followers: {df.loc[df['followers'].idxmax(), 'channel_info']} ({df['followers'].max()/1e6:.1f}M)
   - Highest engagement: {df.loc[df['60_day_eng_rate'].idxmax(), 'channel_info']} ({df['60_day_eng_rate'].max():.1f}%)
   - Top influence score: {df.loc[df['influence_score'].idxmax(), 'channel_info']} ({df['influence_score'].max()})

3. GEOGRAPHIC DISTRIBUTION
   - USA dominates with {(df['country']=='United States').sum()} influencers
   - {df['country'].nunique()} countries represented
   
4. ML RESULTS
   - Regression R²: {r2_rf:.3f}
   - Classification accuracy: {acc*100:.1f}%
""")

In [ ]:
print("\nAll visualizations saved in 'visualizations/' folder")
print("Cleaned data saved in 'data/' folder")